# 1. Expected Goal Model using logistic regression

$\large{\textbf{Keywords:}}$  
$\textbf{RF: }$ Random Forest  
$\textbf{xG: }$ Expected Goals  
$\textbf{ML: }$ Machine Learning  
$\textbf{PLA: }$ Perceptron Learning Algorithm  
$\textbf{SA: }$ Sport Analyst  
$\textbf{kNN: }$ k-nearest neighbor  
$\textbf{GBM: }$ Gradient Boosting Machine  
$\textbf{NB: }$ Native Bayes

### 1.2 Mô hình Logistic Regression
Hai mô hình tuyến tính (Linear Model) **Linear Regression** và (PLA) **Perceptron Learning Algorithm** đều có chung một dạng:

$y = f\left(w^{T} x\right)$

Trong đó hàm $f()$ được gọi là **Activate Function** và $x$ được hiểu là dữ liệu mở rộng với $x_0 = 1$ được thêm vào để thuận tiện cho việc tính toán.
 




In [23]:
import sys
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import math

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as sk_metrics
from sklearn.metrics import precision_score

from preprocessing.event_data import run_pipeline
from supportFolder.plot_pitch import create_soccer_Pitch

In [24]:
#df_events = run_pipeline(save=True)

In [25]:
path_parquet = "/home/datgegt/projects/tf216/football-data-visualization-and-analysis-dashboard/dataset/data_transforms/event_data_transform.parquet"
df_events = pd.read_parquet(path_parquet, engine='pyarrow')

In [26]:
df_events.head(10)

,ID,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerID,playerName,playerPosition,playerStrongFoot,teamPossession,homeTeamID,awayTeamID,Goal,Own goal,Counter attack,bodyPartShot,bodyPartShotCode
0,700820,1694390,2H,0.814000,Pass,Simple pass,11944,52.50,32.64,51.45,32.64,83753,N. Stanciu,Midfielder,right,11944.0,4418,11944,0,0,0,Unknown,0
1,700821,1694390,2H,0.814000,Pass,Simple pass,11944,51.45,32.64,40.95,34.00,6165,F. Andone,Forward,right,11944.0,4418,11944,0,0,0,Unknown,0
2,700001,1694390,1H,1.255990,Pass,Simple pass,4418,52.50,32.64,49.35,34.00,26010,O. Giroud,Forward,left,4418.0,4418,11944,0,0,0,Unknown,0
3,700002,1694390,1H,2.351908,Pass,Simple pass,4418,49.35,34.00,43.05,32.64,3682,A. Griezmann,Forward,left,4418.0,4418,11944,0,0,0,Unknown,0
4,700822,1694390,2H,2.677000,Pass,High pass,11944,40.95,34.00,72.45,9.52,83824,M. Pintilii,Midfielder,right,11944.0,4418,11944,0,0,0,Unknown,0
5,700003,1694390,1H,3.241028,Pass,Simple pass,4418,43.05,32.64,33.60,23.80,31528,N. Kanté,Midfielder,right,4418.0,4418,11944,0,0,0,Unknown,0
6,700823,1694390,2H,5.545170,Pass,Head pass,4418,32.55,58.48,74.55,63.92,7858,B. Sagna,Defender,right,4418.0,4418,11944,0,0,0,Unknown,0
7,700004,1694390,1H,6.033681,Pass,High pass,4418,33.60,23.80,93.45,4.08,7855,L. Koscielny,Defender,right,4418.0,4418,11944,0,0,0,Unknown,0
8,700824,1694390,2H,10.532000,Pass,Launch,11944,30.45,4.08,58.80,4.08,105330,R. Raț,Defender,left,11944.0,4418,11944,0,0,0,Unknown,0
9,700005,1694390,1H,13.143591,Duel,Ground defending duel,4418,93.45,4.08,89.25,0.00,25437,B. Matuidi,Midfielder,left,4418.0,4418,11944,0,0,0,Unknown,0


In [27]:
df_shots = df_events[df_events['eventName'] == 'Shot'].copy()

In [28]:
df_shots[['posBeforeXMeters', 'posBeforeYMeters']].describe()

,posBeforeXMeters,posBeforeYMeters
count,43068.000000,43068.000000
mean,89.079096,33.481963
std,8.222725,9.366742
min,52.500000,0.000000
25%,81.900000,26.520000
50%,91.350000,33.320000
75%,95.550000,40.800000
max,105.000000,68.000000


In [29]:
df_shots

,ID,matchID,matchPeriod,eventSec,eventName,subEventName,teamID,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerID,playerName,playerPosition,playerStrongFoot,teamPossession,homeTeamID,awayTeamID,Goal,Own goal,Counter attack,bodyPartShot,bodyPartShotCode
24,700009,1694390,1H,31.226217,Shot,Shot,4418,95.55,19.72,0.0,0.0,25437,B. Matuidi,Midfielder,left,4418.0,4418,11944,0,0,0,rightFoot,3
61,700851,1694390,2H,100.604872,Shot,Shot,11944,79.80,18.36,105.0,68.0,83753,N. Stanciu,Midfielder,right,11944.0,4418,11944,0,0,0,rightFoot,3
85,700865,1694390,2H,130.592908,Shot,Shot,11944,93.45,27.20,105.0,68.0,33235,B. Stancu,Forward,right,11944.0,4418,11944,0,0,0,rightFoot,3
89,700044,1694390,1H,143.119551,Shot,Shot,11944,74.55,19.72,105.0,68.0,83824,M. Pintilii,Midfielder,right,11944.0,4418,11944,0,0,0,rightFoot,3
137,700060,1694390,1H,219.576026,Shot,Shot,11944,100.80,38.76,105.0,68.0,33235,B. Stancu,Forward,right,11944.0,4418,11944,0,0,0,rightFoot,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024471,2746238,2576338,1H,1794.686641,Shot,Shot,3185,99.75,34.68,105.0,68.0,8148,Iago Falqué,Forward,left,3185.0,3193,3185,1,0,0,leftFoot,1
3024611,2747170,2576338,2H,2065.034482,Shot,Shot,3193,94.50,31.28,0.0,0.0,21177,G. Pandev,Forward,left,3193.0,3193,3185,1,0,0,leftFoot,1
3024707,2747219,2576338,2H,2367.252041,Shot,Shot,3193,82.95,21.76,0.0,0.0,349102,S. Omeonga,Midfielder,right,3193.0,3193,3185,0,0,0,rightFoot,3
3024829,2747287,2576338,2H,2579.867806,Shot,Shot,3193,97.65,29.24,0.0,0.0,21177,G. Pandev,Forward,left,3193.0,3193,3185,0,0,0,leftFoot,1


In [30]:
total_shots = len(df_shots)
print(f"Tổng số cú sút: {total_shots}")
print(f"Xác suất ghi bàn khi thực hiện cú sút: {df_shots['Goal'].mean()*100:.2f}%")

Tổng số cú sút: 43068
Xác suất ghi bàn khi thực hiện cú sút: 10.43%


## TRAIN MODEL WITH LOGISTIC REGRESSION

In [31]:
feature_cols = ["posBeforeXMeters", "posBeforeYMeters", "bodyPartShotCode", "Counter attack"]
target_col = ["Goal"]

X = df_shots[feature_cols]
y = df_shots[target_col]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [33]:
reg_shitty = LogisticRegression(random_state=42)
reg_shitty.fit(X_train, np.array(y_train).ravel())

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [34]:
pred_vals = reg_shitty.predict(X_test)

In [35]:
print(f"Accuracy of the model is {sk_metrics.accuracy_score(y_test, pred_vals)*100:.1f}%")

Accuracy of the model is 89.4%


In [36]:
for i, col in enumerate(X_train.columns):
    print(f"Coefficient of {col}: {reg_shitty.coef_[0][i]:.3f}")

Coefficient of posBeforeXMeters: 0.128
Coefficient of posBeforeYMeters: -0.000
Coefficient of bodyPartShotCode: 0.016
Coefficient of Counter attack: 0.594


<b>posOrigXMeters</b>: Với mỗi vị trí sút theo trục X tăng thêm 1 mét, giá trị dự đoán của mô hình giảm trung bình đi 0.13%. Điều này phản anh rằng các cầu thủ càng xa khung thành theo trục X, khả năng thành công càng giảm.  
<b>posOrigYMeters</b>: khi vị trí cầu thủ sút lệch sang trái/phải thêm 1 mét, giá trị dự đoán giảm 0.04%.mức ảnh hưởng nhỏ hơn so với trục X.  
<b>bodyPartShotCode</b>: Bộ phận cơ thể có giá trị âm điều này phản ánh rằng bộ phận cơ thể làm giảm khả năng ghi bàn.  
<b>CounterAttack</b>: Các pha phản công có xác suất sút cao hơn bằng cút sút nguy hiểm/ghi bàn cao hơn với xác suất được mô hình đánh giá là 45.12%.  


#### Log loss

In [37]:
pred_probs = reg_shitty.predict_proba(X_test)[:,1]
print(f"Log loss of our model: {sk_metrics.log_loss(y_test, pred_probs):.5f}")

Log loss of our model: 0.30154


In [38]:
print(f"Log loss of dummy: {sk_metrics.log_loss(y_test, [0.1056]*len(y_test)):.3f}")

Log loss of dummy: 0.337


Với *Log loss = 0.33510*, mô hinh cho thấy đạt mức độ dự đoán khá tốt so với bộ dữ liệu có tỉ lệ ghi bàn thấp *(10.42%)*. Giá trị này cho thấy mô hình đang tạo ra xác suất dự đoán tương đối chính xác và ổn định, mặc dù vẫn chưa vượt nhiều mong đợi so với baseline xác suất trung bình.

#### AUC

In [39]:
# compute the AUC of our shitty model
print(f"AUC of our model: {sk_metrics.roc_auc_score(y_test, pred_probs)*100:.2f}%")

AUC of our model: 74.19%


In [40]:
import numpy as np
from scipy.special import logit, expit

# tránh logit(0) hoặc logit(1)
eps = 1e-15
pred_probs_clipped = np.clip(pred_probs, eps, 1 - eps)

logits = logit(pred_probs_clipped)
logits_over = logits * 6      # hệ số overconfidence
pred_probs_over = expit(logits_over)

print(f"Currently predicted success when shooting: {np.mean(pred_probs)*100:.2f}%")
print(f"Overestimated predicted success when shooting: {np.mean(pred_probs_over)*100:.2f}%")

Currently predicted success when shooting: 10.34%
Overestimated predicted success when shooting: 0.05%


In [41]:
print(f"Log loss w/o overestimation: {sk_metrics.log_loss(y_test, pred_probs):.3f}")
print(f"Log loss with overestimation: {sk_metrics.log_loss(y_test, pred_probs_over):.3f}")
print(f"AUC w/o overestimation: {sk_metrics.roc_auc_score(y_test, pred_probs)*100:.2f}%")
print(f"AUC with overestimation: {sk_metrics.roc_auc_score(y_test, pred_probs_over)*100:.2f}%")

Log loss w/o overestimation: 0.302
Log loss with overestimation: 1.131
AUC w/o overestimation: 74.19%
AUC with overestimation: 74.19%


#### Heatmap show the location shots taken on pitch 

In [42]:
pitch = create_soccer_Pitch(theme="tactical")

metrics = {"Number of shots": {"col": None, "agg": "count"}}


shots_data, x, y, = pitch.add_binned_heatmap(df_shots,
    col_x='posBeforeXMeters',
    col_y='posBeforeYMeters',
    nb_buckets_x=24,
    nb_buckets_y=17,
    metrics=metrics
)

nb_shots = shots_data["Number of shots"]
total_shots = nb_shots.sum()

share_shots = (nb_shots/total_shots*100) if total_shots > 0 else nb_shots

hover_info = {
    "Shots": {"values": nb_shots, "display_type": ".0f"},
    "Share of shots (%)": {"values": share_shots, "display_type": ".2f"},
    }


pitch.add_heatmap_(
    z = share_shots,
    x_centers = x,
    y_centers = y,
    title="Shots Location Distribution (%)",
    dict_infor=hover_info,
    opacity=0.9
)


pitch.show()

**Note:** Dữ liệu bóng đá wyscout trên đã được flip sân. Mục đích giúp cho việc dự đoán các tỉ số như (xG, Goal, AUC, Log Loss) đạt hiệu suất cao, tránh việc mô hình học các vị trí sút ở cả hai cầu môn làm giảm hiệu suất đánh giá mô hình và mô hình khó đạt được điểm số cao trong **Phân tích bóng đá**  

**Biểu đồ nhiệt cập nhật các vị trí sút bóng và tỉ trọng cú sút bóng:**  
Công thức tính Share of Shots (Tỷ trọng cú sút)

$$S_{xy} = \frac{n_{xy}}{N_{total}} \times 100$$

**Trong đó:**
- $S_{xy}$: Share of shots tại vị trí $(x, y)$.
- $n_{xy}$: Số cú sút tại vị trí $(x, y)$.
- $N_{total}$: Tổng số cú sút trên toàn sân ($N_{total} = \sum n$).

1. Khu vực trong cầu môn là khu vực có nhiều cú sút nhất. Với tỉ trọng sút đạt gần 4.03-2.49%.  
2. Tiếp đến là các khu vực trước điểm chấm phạt đền giao điểm với đường cầu môn. Biểu diễn màu sắc sáng nhất trên sân bóng


#### Heatmap calculate the number of  goals player taken on pitch

In [43]:
pitch.fig.data = []

pitch_prob = create_soccer_Pitch(theme="tactical")

df_goals = df_shots[df_shots["Goal"] == 1].copy()

metrics = {"Number of goals": {"col": None, "agg": "count"}}

goals_data, x, y = pitch.add_binned_heatmap(
    df_goals,
    col_x='posBeforeXMeters',
    col_y='posBeforeYMeters',
    nb_buckets_x=24,
    nb_buckets_y=17,
    metrics=metrics
    )

nb_goals = goals_data["Number of goals"]

goal_probability = np.divide(
    nb_goals,
    nb_shots,
    out=np.zeros_like(nb_goals, dtype=float),
    where=nb_shots != 0
) * 100

dict_infor = {f"Scoring probability (%)": {"values": goal_probability, "display_type": ".2f"},
              f"Share of shots (%)": {"values": share_shots, "display_type": ".2f"},
              f"Number of shots": {"values": nb_shots, "display_type": ".0f"},
              f"Number of goals": {"values": nb_goals, "display_type": ".0f"}}

fig = pitch_prob.add_heatmap_(goal_probability, x, y, dict_infor=dict_infor, title="Observed Goal Conversion Rate by Shot Location (%)", opacity=0.9)
fig.show()


$\large{\textbf{\text{Đánh giá lại biểu đồ heatmap}}}$  
- Có một số vị trí được đánh giá tỉ lệ sút bóng ghi bàn cao nhất. Ví dụ như góc phải bên dưới đường cầu môn đưa ra một tỉ lệ ảo(chỉ có 1 shot và 1 goal) nên việc tính toán trên biểu đồ nhiệt này đưa ra tỉ lệ goal ở đây là 100%.
- Một số vị trí gần đường giữa sân và ở cột cờ góc(góc trên bên trái) với tỉ lệ goal vào khoảng 33%-50% nhưng thực sự số lượng shot ở các vị trí này không nhiều và số lượng goal xấp xỉ với tỉ lệ sút bóng.  

=> $\textbf{Hướng giải quyết:}$ Khắc phục các điểm dữ liệu xa vị trí khung thành nhưng tỉ lệ goal cao ở các vị trí xa khung thành và các vị trí góc với số lượng shot thấp nhưng tỉ lệ goal cao

$\large{\textbf{\text{Đánh giá xác suất shot trên từng ô bucket}}}$  

In [44]:
pitch.fig.data = []

pitch_prob_shoot = create_soccer_Pitch(theme="tactical")

df_passed = df_events[df_events["eventName"] == "Pass"].copy()


metrics_pass = {"Number of passes": {"col": None, "agg": "count"}}

passes_data, x, y = pitch.add_binned_heatmap(
    df_passed,
    col_x='posBeforeXMeters',
    col_y='posBeforeYMeters',
    nb_buckets_x=24,
    nb_buckets_y=17,
    metrics=metrics_pass
    )

nb_passes = passes_data["Number of passes"]


number_of_passesShots = nb_passes + nb_shots

shots_probability = np.divide(
    nb_shots,
    number_of_passesShots,
    out=np.zeros_like(nb_shots, dtype=float),
    where=number_of_passesShots != 0
) * 100

dict_infor = {f"Probability to shoot (%)": {"values": shots_probability, "display_type": ".2f"},
              f"Number of shots": {"values": nb_shots, "display_type": ".0f"},
              f"Number of passes": {"values": nb_passes, "display_type": ".0f"}}

fig = pitch_prob_shoot.add_heatmap_(shots_probability, x, y, dict_infor=dict_infor, title="Shooting Probability per Action (%)", opacity=0.9)
fig.show()



$\large{\textbf{\text{Đánh giá xác suất của mỗi cú sút thành công}}}$  
- Biểu đồ nhiệt về xác suất các cú sút thành công trên các bucket đánh giá trực quan hơn ở các vị trí gần các khu vực ở cầu môn với tỉ lệ cao nằm trong vùng 76% đến 96%.  
- Khu vực gần chấm phạt đền cũng chiếm xác suất rất cao(67.14%)
- So với xác suất ghi bàn thắng dựa trên cú sút tại các điểm chia buckets thì xác suất cú sút dựa trên tỉ lệ sút thành công có vẻ biểu diễn tốt hơn và phù hợp logic của bóng đá hơn. Các vị trí sút nên gần hơn với khung thành, điều này phản ánh đúng hơn những cú sút ghi bàn ở góc hoặc xa khung thành(nhằm tránh mô hình hiểu sai việc một khoảng cách sút xa khung thành hơn hoặc góc sút lớn hơn so với khung thành nhưng tỉ lệ sút lại cao hơn việc khoảng cách gần khung thành và góc sút nhỏ hơn nhưng tỉ lệ sút trúng cao hơn).
- Kiểm tra này nhằm loại bỏ những điểm dữ liệu gây nhiễu cho mô hình và giúp người dùng đánh giá được dựa trên hình ảnh trực quan có thể dễ hiểu hơn.

